<a href="https://colab.research.google.com/github/DavoodSZ1993/RL/blob/main/04_MC_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import gym

env= gym.make('Blackjack-v0')
env.action_space, env.action_space.sample()  # two actions: 0: stay, 1: hit

(Discrete(2), 1)